In [361]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [362]:
from scipy import stats
from scipy.stats import norm, skew

In [363]:
import seaborn as sb

In [546]:
#EDA

In [547]:
#possible Missing values
miss_val=['NaN']

In [548]:
# Reading Data

In [549]:
aisles=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/aisles.csv', na_values=miss_val)

In [550]:
departments=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/departments.csv', na_values=miss_val)

In [551]:
products=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/products.csv', na_values=miss_val)
#products.isnull().values.any()
#products.head(10)

In [552]:
order_products_prior=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__prior.csv', na_values=miss_val)

In [553]:
order_products_train=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/order_products__train.csv', na_values=miss_val)

In [554]:
orders=pd.read_csv('/Users/taimoorjaved/Desktop/Dev/instacart-market-basket-analysis-files/orders.csv', na_values=miss_val)
orders.fillna(-1, inplace=True) # Fill missing values with -1

In [555]:
# Combining dataframes orders_prior and orders_train
order_products_total=pd.concat([order_products_prior, order_products_train], axis=0)
order_products_total.shape

(33819106, 4)

In [ ]:
order_products_usr=pd.merge(orders, order_products_total, on='order_id')
#order_products_usr.isnull().sum()
order_products_usr.head(10)

In [ ]:
#Most ordered product by all users
most_freq_prod=order_products_usr.groupby('product_id').size().sort_values(ascending=False)
print(most_freq_prod)

In [426]:
# Total number of times each product was bought by every user
freq_user_product=dict()

for i in range(1,2): #for i in range (1, order_products_usr['user_id'].max()+1):
    freq_user_product=order_products_usr.groupby('user_id')['product_id'].value_counts()
    #freq_user_product[i]=order_products_usr[order_products_usr['user_id']==i]['product_id'].value_counts()
    #most_freq_product_user[i]=freq_user_product[i].index[0]
    #print("Most frequent product for User {} is {}\n\n".format(i,most_freq_product_user[i]))

print(freq_user_product)

user_id  product_id
1        196           11
         10258         10
         12427         10
         25133          9
         13032          4
                       ..
206209   41665          1
         42606          1
         44325          1
         48370          1
         48697          1
Name: product_id, Length: 13863746, dtype: int64


In [474]:
# Total number of unique orders for every user
total_orders_user=order_products_usr.groupby('user_id')['order_id'].nunique()
print(total_orders_user)

user_id
1         11
2         15
3         12
4          5
5          5
          ..
206205     4
206206    67
206207    16
206208    49
206209    14
Name: order_id, Length: 206209, dtype: int64


In [459]:
# This gives the total number of products in EACH order against all users.
# Segregation of the below values.
total_orders_user3=order_products_usr.groupby('user_id')['order_id'].value_counts() 
print(total_orders_user3)

user_id  order_id
1        1187899     11
         2550362      9
         431534       8
         2295261      6
         2398795      6
                     ..
206209   272231       8
         1854736      8
         2307371      3
         2558525      3
         3186442      2
Name: order_id, Length: 3346083, dtype: int64


In [475]:
# This gives the total number of products in all orders for all users. Sum of values above
total_orders_user2=order_products_usr.groupby('user_id')['order_id'].size() 
print(total_orders_user2)

user_id
1          70
2         226
3          88
4          18
5          46
         ... 
206205     51
206206    285
206207    223
206208    677
206209    137
Name: order_id, Length: 206209, dtype: int64


In [432]:
torders=0
for key, value in total_orders_user3.items():
    if (key[0]==1):
        torders+=value

print(torders)

70


In [424]:
#Number of unique products ordered by every user
unique_product_user=dict()

for i in range(1,2): #for i in range (1, order_products_usr['user_id'].max()+1):
    unique_product_user=order_products_usr.groupby('user_id')['product_id'].nunique()

print(unique_product_user)    

user_id
1          19
2         121
3          33
4          17
5          28
         ... 
206205     37
206206    150
206207     92
206208    198
206209     72
Name: product_id, Length: 206209, dtype: int64


In [492]:
#new_product_ratio for every user= unique products/total products
nd=pd.merge(unique_product_user, total_orders_user2, on="user_id")
nd.columns=['unique_product_count', 'total_product_count']
nd['new_product_ratio'] = nd.apply(lambda row: row.unique_product_count/row.total_product_count*100, axis=1)
print(nd)

         unique_product_count  total_product_count  new_product_ratio
user_id                                                              
1                          19                   70          27.142857
2                         121                  226          53.539823
3                          33                   88          37.500000
4                          17                   18          94.444444
5                          28                   46          60.869565
...                       ...                  ...                ...
206205                     37                   51          72.549020
206206                    150                  285          52.631579
206207                     92                  223          41.255605
206208                    198                  677          29.246677
206209                     72                  137          52.554745

[206209 rows x 3 columns]


In [502]:
#Product ordering ratio for every user = total number of orders containing product/ total number of orders
print(order_products_usr.groupby('user_id')['order_id'].size())

user_id
1          70
2         226
3          88
4          18
5          46
         ... 
206205     51
206206    285
206207    223
206208    677
206209    137
Name: order_id, Length: 206209, dtype: int64


In [471]:
# Frequency of products ordered by every user in each order
user_order_products=order_products_usr.groupby(['user_id', 'order_id'])['product_id'].value_counts()
print(user_order_products)

user_id  order_id  product_id
1        431534    196           1
                   10258         1
                   10326         1
                   12427         1
                   13176         1
                                ..
206209   3154581   40396         1
                   40534         1
                   41213         1
         3186442   6187          1
                   14197         1
Name: product_id, Length: 33819106, dtype: int64


In [499]:
# Most Frequent time for every user
user_time=order_products_usr.groupby('user_id')['order_hour_of_day'].value_counts()

In [498]:
#Most frequent Time of purchase for the shop
most_popular_time=order_products_usr.groupby('order_hour_of_day').size()

In [497]:
# Most frequent day of purchase for every user
user_day=order_products_usr.groupby('user_id')['order_dow'].value_counts()

In [495]:
# Most frequent day of purchase for shop
most_popular_day=order_products_usr.groupby('order_dow').size()

In [503]:
#How many times a user has reordered a product
user_products_NotReordered=order_products_usr.groupby(['user_id', 'product_id'])['reordered'].aggregate("sum")
print (user_products_NotReordered)

user_id  product_id
1        196           10
         10258          9
         10326          0
         12427          9
         13032          3
                       ..
206209   43961          2
         44325          0
         48370          0
         48697          0
         48742          1
Name: reordered, Length: 13863746, dtype: int64


In [386]:
#Total number of times a product has been reordered.
#percentage of user wrt product

In [388]:
#Overall percentage of reorders for each product

In [389]:
#Most ordered product in every department

In [390]:
#Did any user reorder their previous order

In [391]:
#Most frequent time when users purchase

In [ ]:
#Times a customer orders from the same department. Wrt to customer

In [ ]:
#Days a customer orders the same products

In [ ]:
#Times a product is ordered from the same department. Wrt product